# data analysis task

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict, cross_val_score

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

from scipy import stats

## Read in the data

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
# Some basic cleaning as column headers have trailing whitespace, clear these up
print(f"Before: {df.columns}")
df.columns = [c.strip() for c in df.columns]
print(f"After: {df.columns}")

## Test/train split to avoid peaking during exploration

In [ ]:
# specify target column name
target_col = 'Out'


In [ ]:
train_cols = [c for c in df.columns if c != target_col]

df_train_features, df_test_features, df_train_target, df_test_target = train_test_split(
    df[train_cols], 
    df[target_col], 
    test_size=0.2,
    random_state=1,
    shuffle=True,
)

## Initial exploration

In [ ]:
df_train_features.sample(3)

In [ ]:
# get an overview of descriptive statistics of the data
df_train_features.describe()

In [ ]:
# Check for null values in an cell
df_train_features.isnull().any()

## Make some descriptive plots

### Features

In [ ]:
plt.figure(figsize=(15,9))
i = 0
for col in df_train_features.columns:
    i+=1
    plt.subplot(3,4,i)
    plt.plot(df_train_features[col], ".")
    plt.title(col)
    if i == 1:
        plt.ylabel('raw plots')
    
    plt.subplot(3,4,i+4)
    plt.plot(np.sort(df_train_features[col]), ".")
    if i+4 == 5:
        plt.ylabel('sorted')
    
    plt.subplot(3,4,i+8)
    plt.hist(df_train_features[col])
    if i+8 == 9:
        plt.ylabel('histograms')

plt.tight_layout()
plt.show()

In [ ]:
print(f"% of rows with In2 -1 value: {(df_train_features['In2'] == -1).mean() * 100}%")

### Target

In [ ]:
plt.figure(figsize=(7,2))

plt.subplot(1,3,1)
plt.plot(np.sort(df_train_target), ".")

plt.subplot(1,3,2)
plt.plot(np.sort(df_train_target), ".")
plt.yscale('log')

plt.subplot(1,3,3)
plt.hist(df_train_target,bins=10)
plt.yscale('log')

plt.tight_layout()
plt.show()

In [ ]:
print(f"% of rows with outlier target: {(df_train_target > 10000).mean() * 100}%")

## Comments

- Features In1 and In4 are binary
- In3 seems to follow a uniform distribution
- In2 follows also follows a uniform distribution with ~2% of values as -1.
- The target column has a strange distribution with 75% following a logarithmic growth (when plotted in order) with a step and then a few (~4) outliers

Without knowing more about the data and what is being measured it is difficult to know how to treat the outliers in the target. My experience would say that these are either errors in the measuring method and these rows should be discarded. However, it's possible that these are valid, but rare events which may be the most important to predict.

Likewise, the In2 values with -1: My instinct is that these are errors in a sensor, however they could be meaningful values that are indicative of a non-linearity.


## Look at the relationship between plots

In [ ]:
plt.figure(figsize=(7,7))
sns.pairplot(
    pd.merge(
        df_train_features,
        np.log10(df_train_target),
        left_index=True,
        right_index=True
    ),
    height=1.5)
plt.show()

### Comments

- Doesn't seem to be much correlation between any of the features, so no need to remove features or look for principal components / other dimensionality reduction methods
- In2 does seem to have some relationship with the 'logarithmic' growth portion of the target
- It doesn't look like the -1 In2 values are predictive of the Out outlier values
- Feature scaling of In2 and In3 to get these into the same range may be beneficial

## Modelling Approach

The modelling approach crucially depends on how we want to treat the outliers in the target.

_Scenario 1_: If we believe them to be erroneous then we can just remove any rows where the 'Out' value is greater than 10,000 (in both the training and test sets).  

_Scenario 2_: If we believe that these are true values then I think that a linear model will really struggle to fit the data. We can do a few things such as fitting the log of the 'Out' values as opposed to the true values, trying MAE as well as MSE, but ultimately I think this big non-linearity will require a non-linear model such as a random forest. Most likely I would approach the problem with an ensemble approach:
- classification/outlier detection
- regression on non-outlier data

In the brief it said that the the data has not been preprocessed and that the data can be fit without advanced machine learning algorithms. This makes me lean towards assuming that the data has erroneous outliers that should be removed. So I am going to make this assumption here.

Likewise, we can either treat the In2 values that are equal to -1 as indicative of bad data and just remove these rows, or we can treat these as a feature - perhaps creating a new binary feature that acts as an idicator for a -1 value and perhaps capture more of the non-linearity aspect of this feature allowing for the linear values to be considered as they are. 

## Feature Engineering

Don't know much about these features, but I think it would be interesting to see if creating more features that capture their interactions may be additionally predcitive.

I can also create a feature that acts as an indicator for -1 values in 'In2'. 

Feature scaling is probably also a good idea as In2 is an order of magnitude less than In3 and the other two features are discrete 0-1 values. In2 and In3 are both uniformly distributed so a min-max scaler seems more appropriate than normalizing.


## Feature / Target engineering

In [ ]:
# Get interaction features

def make_interaction_features_df(df_in):
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    interactions = poly.fit_transform(df_in)

    n_original_features = df_in.shape[1]
    df_interactions = pd.DataFrame(
        data=interactions[:,n_original_features:], 
        index=df_in.index,
        columns=poly.get_feature_names_out()[n_original_features:]
    )
    
    return df_interactions


In [ ]:
# Create boolean feature for the In2 -1 value

def make_In2_non_linear_bool_feature_df(df_in):
    
    bool_vector = df_in['In2'] < 0
    
    return pd.DataFrame(bool_vector).rename({'In2': 'In2_bool'}, axis=1)

In [ ]:
# remove the rows from target and features with bad out rows

def remove_bad_rows_out(df_features_in, df_target_in, threshold):
    
    good_idx = df_target_in.loc[df_target_in < 10000].index.tolist()
    
    df_features_out = df_features_in.loc[good_idx].copy()
    df_target_out = df_target_in.loc[good_idx].copy()
    
    return df_features_out, df_target_out


In [ ]:
# remove rows from features and target with bad In2 values

def remove_bad_In2_rows(df_features_in, df_target_in):
    
    good_idx = df_features_in.loc[df_features_in['In2'] != -1].index.tolist()
    
    df_features_out = df_features_in.loc[good_idx].copy()
    df_target_out = df_target_in.loc[good_idx].copy()
    
    return df_features_out, df_target_out

## Modelling

Run a set of experiments with several models and optional preprocessing and feature engineering steps.

In [ ]:
lin_reg = LinearRegression()
las_reg = Lasso()
rf_reg = RandomForestRegressor()

options = {
    'model': [
        lin_reg, 
        las_reg, 
        rf_reg,
    ],
    'data_cleaning': [
        'none',
        'remove_bad_In2_rows'
    ],
    'scaling': [
        True,
        False,
    ],
    'additional_features': [
        'none',
        'interaction',
        'In2_bool',
        'interaction,In2_bool',
    ],
}

In [ ]:
experiments = []
for model in options['model']:
    for clean in options['data_cleaning']:
        for scaling in options['scaling']:
            for af in options['additional_features']:

                if ('In2_bool' in af) and (clean == 'remove_bad_In2_rows'):
                    #skip this iteration
                    continue

                df_train_f = df_train_features.copy()
                df_train_t = df_train_target.copy()

                # remove bad out rows
                df_train_f, df_train_t = remove_bad_rows_out(df_train_f, df_train_t, 10000)

                if clean == 'remove_bad_In2_rows':
                    df_train_f, df_train_t = remove_bad_In2_rows(df_train_f, df_train_t)


                if scaling:
                    scaler = MinMaxScaler()
                    df_train_f.loc[:,['In2','In3']] = scaler.fit_transform(
                        df_train_f[['In2','In3']]
                    )

                if 'interaction' in af:

                    df_train_f = pd.merge(
                        df_train_f,
                        make_interaction_features_df(df_train_f),
                        left_index=True,
                        right_index=True,
                    )

                if 'In2_bool' in af:

                    df_train_f = pd.merge(
                        df_train_f,
                        make_In2_non_linear_bool_feature_df(df_train_f),
                        left_index=True,
                        right_index=True,
                    )

                try:
                    predictions = cross_val_predict(
                        model, 
                        df_train_f, 
                        df_train_t, 
                        cv=10,
                        verbose=0,
                    )

                    experiments += [{
                        'model': model,
                        'cleaning': clean,
                        'scaling': scaling,
                        'additional_features': af,
                        'rmse': mean_squared_error(df_train_t, predictions, squared=False)
                        # 'rmse_transformed_target': mean_squared_error(np.log10(df_train_target), predictions, squared=False),
                        # 'rmse': mean_squared_error(df_train_target, (10**predictions), squared=False)
                    }]
                except:
                    pass
        

In [ ]:
pd.DataFrame(experiments).sort_values(['cleaning','rmse'])

# Comments

Naturally we can't directly compare the cleaned and non-cleaned model performances due to differing number of rows, and the fact that the non-cleaned data we have a priori reason to believe may be harder due to having erroneous data.

Here I'm going to focus on the models with the -1 In2 values removed, as the results makes me believe that this data was indeed erroneous as the RMSE is very low when they are not present in the data.

It is interseting to note however that the models that performed best when including these values was those with the In2_bool indicator feature - which likely provided the model with the signal to treat these rows differently.

It appears that the feature scaling has little to no effect, even a detrimental effect, so this preprocessing step can be ommitted.

The models with interaction features consistently outperformed those without them, so these should be included.

It may be that the RandomForestRegressor could do better with better hyperparameters, but with the linear model already performing well, and this being a much easier to explain model, I will progress with this.

## Deeper dive into the best performing model

In [ ]:
# Preprocessing

df_train_f = df_train_features.copy()
df_train_t = df_train_target.copy()

df_train_f, df_train_t = remove_bad_rows_out(df_train_f, df_train_t, 10000)
df_train_f, df_train_t = remove_bad_In2_rows(df_train_f, df_train_t)

df_train_f = pd.merge(
    df_train_f,
    make_interaction_features_df(df_train_f),
    left_index=True,
    right_index=True,
)

In [ ]:
# Cross Validation Scores to check overfitting

scores = -cross_val_score(
    lin_reg, 
    df_train_f, 
    df_train_t, 
    cv=10,
    scoring='neg_root_mean_squared_error',
    verbose=0,
)

print(f"mean rmse: {np.mean(scores)}")
print(f"sd rmse: {np.std(scores)}")

##### Comments

This distribution of scores suggests that the model is fairly robust to overfitting as the variance is quite low

#### Now build model with a single test train split

In [ ]:
df_train_x, df_val_x, df_train_y, df_val_y = train_test_split(
    df_train_f, 
    df_train_t, 
    test_size=0.2,
    random_state=1,
    shuffle=True,
)

lin_reg.fit(df_train_x, df_train_y)
predictions = lin_reg.predict(df_val_x)

In [ ]:
def plot_prediction_performance(predictions, true_vals):
    
    plt.figure(figsize=(8,2))

    plt.subplot(1,3,1)
    plt.scatter(predictions, true_vals, s=2, alpha=.3);
    plt.xlabel('prediction')
    plt.ylabel('true value')
    plt.title('prediction v target')


    plt.subplot(1,3,2)
    plt.scatter(
        np.arange(len(predictions)),
        (predictions - true_vals), 
        s=1,
    )
    plt.plot(
        [0,len(predictions)],
        [0,0],
        color='grey'
    )
    plt.xlabel('prediction')
    plt.ylabel('residual')
    plt.title('residuals')


    plt.subplot(1,3,3)
    plt.scatter(
        np.arange(len(predictions)),
        np.sort((predictions - true_vals)), 
        s=1,
    )
    plt.plot(
        [0,len(predictions)],
        [0,0],
        color='grey'
    )
    plt.xlabel('prediction')
    plt.ylabel('residual')
    plt.title('residuals - ordered')

    plt.tight_layout()
    plt.show()

In [ ]:
# Plot the results

plot_prediction_performance(predictions, df_val_y)

##### Comments

As indicated previously by the RMSE, the first chart shows that this model does seem to fit the data very well and the remaining residual looks like it could be noise.

## Assessing performance on the hold-out test set

In [ ]:
# preprocessing
 
## Preprocessing training data

df_train_f = df_train_features.copy()
df_train_t = df_train_target.copy()

df_train_f, df_train_t = remove_bad_rows_out(df_train_f, df_train_t, 10000)
df_train_f, df_train_t = remove_bad_In2_rows(df_train_f, df_train_t)

df_train_f = pd.merge(
    df_train_f,
    make_interaction_features_df(df_train_f),
    left_index=True,
    right_index=True,
)


# Preprocessing test data

df_test_f = df_test_features.copy()
df_test_t = df_test_target.copy()

df_test_f, df_test_t = remove_bad_rows_out(df_test_f, df_test_t, 10000)
df_test_f, df_test_t = remove_bad_In2_rows(df_test_f, df_test_t)

df_test_f = pd.merge(
    df_test_f,
    make_interaction_features_df(df_test_f),
    left_index=True,
    right_index=True,
)

# Model fitting

lin_reg.fit(df_train_f, df_train_t)
predictions = lin_reg.predict(df_test_f)


# Plot the results

plot_prediction_performance(predictions, df_test_t)

### Comments

Overall it looks like the model has performed well on the test set and the spread of the residuals is very similar to in training, again suggesting a model that has low variance and with low bias as the residuals seem centred around 0.

In [ ]:
# Model stats from the training set

df_train_f2 = sm.add_constant(df_train_f)
est = sm.OLS(df_train_t, df_train_f2)
est2 = est.fit()
print(est2.summary())

##### Comments

The model stats seem to point towards In2 and the interaction of In1 and In4 (the two binary features) being the most important features in the model (as well as the constant term). 